In [1]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize
from random import randrange
from sklearn.svm import SVC

def preprocess():
    """ 
     Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.

     Output:
     train_data: matrix of training set. Each row of train_data contains 
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data 
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the 
       training set
     test_data: matrix of training set. Each row of test_data contains 
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set
    """

    mat = loadmat('mnist_all.mat')  # loads the MAT object as a Dictionary

    n_feature = mat.get("train1").shape[1]
    n_sample = 0
    for i in range(10):
        n_sample = n_sample + mat.get("train" + str(i)).shape[0]
    n_validation = 1000
    n_train = n_sample - 10 * n_validation

    # Construct validation data
    validation_data = np.zeros((10 * n_validation, n_feature))
    for i in range(10):
        validation_data[i * n_validation:(i + 1) * n_validation, :] = mat.get("train" + str(i))[0:n_validation, :]

    # Construct validation label
    validation_label = np.ones((10 * n_validation, 1))
    for i in range(10):
        validation_label[i * n_validation:(i + 1) * n_validation, :] = i * np.ones((n_validation, 1))

    # Construct training data and label
    train_data = np.zeros((n_train, n_feature))
    train_label = np.zeros((n_train, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("train" + str(i)).shape[0]
        train_data[temp:temp + size_i - n_validation, :] = mat.get("train" + str(i))[n_validation:size_i, :]
        train_label[temp:temp + size_i - n_validation, :] = i * np.ones((size_i - n_validation, 1))
        temp = temp + size_i - n_validation

    # Construct test data and label
    n_test = 0
    for i in range(10):
        n_test = n_test + mat.get("test" + str(i)).shape[0]
    test_data = np.zeros((n_test, n_feature))
    test_label = np.zeros((n_test, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("test" + str(i)).shape[0]
        test_data[temp:temp + size_i, :] = mat.get("test" + str(i))
        test_label[temp:temp + size_i, :] = i * np.ones((size_i, 1))
        temp = temp + size_i

    # Delete features which don't provide any useful information for classifiers
    sigma = np.std(train_data, axis=0)
    index = np.array([])
    for i in range(n_feature):
        if (sigma[i] > 0.001):
            index = np.append(index, [i])
    train_data = train_data[:, index.astype(int)]
    validation_data = validation_data[:, index.astype(int)]
    test_data = test_data[:, index.astype(int)]

    # Scale data to 0 and 1
    train_data /= 255.0
    validation_data /= 255.0
    test_data /= 255.0

    return train_data, train_label, validation_data, validation_label, test_data, test_label


def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def blrObjFunction(initialWeights, *args):
    """
    blrObjFunction computes 2-class Logistic Regression error function and
    its gradient.

    Input:
        initialWeights: the weight vector (w_k) of size (D + 1) x 1
        train_data: the data matrix of size N x D
        labeli: the label vector (y_k) of size N x 1 where each entry can be either 0 or 1 representing the label of corresponding feature vector

    Output:
        error: the scalar value of error function of 2-class logistic regression
        error_grad: the vector of size (D+1) x 1 representing the gradient of
                    error function
    """
    train_data, labeli = args

    n_data = train_data.shape[0]
    n_features = train_data.shape[1]

    intercept = np.ones((n_data, 1))
    x = np.hstack((intercept, train_data))
    h = sigmoid(np.dot(x, initialWeights)).reshape(n_data, 1)
    y = labeli

    error = -sum(y * np.log(h) + (1 - y) * np.log(1 - h)) / n_data
    error_grad = np.dot(x.T, (h - y)).reshape(n_features + 1, ) / n_data

    return error, error_grad


def blrPredict(W, data):
    """
     blrObjFunction predicts the label of data given the data and parameter W
     of Logistic Regression

     Input:
         W: the matrix of weight of size (D + 1) x 10. Each column is the weight
         vector of a Logistic Regression classifier.
         X: the data matrix of size N x D

     Output:
         label: vector of size N x 1 representing the predicted label of
         corresponding feature vector given in data matrix

    """

    n_data = data.shape[0]

    intercept = np.ones((n_data, 1))
    x = np.hstack((intercept, data))

    label = sigmoid(np.dot(x, W))

    return np.argmax(label, axis=1)

In [2]:
"""
Script for Logistic Regression
"""
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

# number of classes
n_class = 10

# number of training samples
n_train = train_data.shape[0]

# number of features
n_feature = train_data.shape[1]

Y = np.zeros((n_train, n_class))
for i in range(n_class):
    Y[:, i] = (train_label == i).astype(int).ravel()

In [3]:
# Logistic Regression with Gradient Descent
W = np.zeros((n_feature + 1, n_class))
initialWeights = np.zeros((n_feature + 1, 1))
opts = {'maxiter': 100}
for i in range(n_class):
    print(i)
    labeli = Y[:, i].reshape(n_train, 1)
    args = (train_data, labeli)
    nn_params = minimize(blrObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)
    W[:, i] = nn_params.x.reshape((n_feature + 1,))
    
# notice the use of flatten()

predicted_label = blrPredict(W, train_data)
print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label.flatten()))) + '%')

# Find the accuracy on Validation Dataset
predicted_label = blrPredict(W, validation_data)
print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label.flatten()))) + '%')

# Find the accuracy on Testing Dataset
predicted_label = blrPredict(W, test_data)
print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label == test_label.flatten()))) + '%')

0
1
2
3
4
5
6
7
8
9

 Training set Accuracy:92.724%

 Validation set Accuracy:91.45%

 Testing set Accuracy:92.0%


In [ ]:
"""
Script for Support Vector Machine
"""

print('\n\n--------------SVM-------------------\n\n')

print("Linear")
clf = SVC(kernel='linear')
clf.fit(train_data, train_label)

predicted_label = clf.predict(train_data)
print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label.flatten()))) + '%')

# Find the accuracy on Validation Dataset
predicted_label = clf.predict(validation_data)
print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label.flatten()))) + '%')

# Find the accuracy on Testing Dataset
predicted_label = clf.predict(test_data)
print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label == test_label.flatten()))) + '%')



--------------SVM-------------------


Linear


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 Training set Accuracy:97.286%

 Validation set Accuracy:93.64%

 Testing set Accuracy:93.78%


In [ ]:
print("Radial; Gamma: 1")
clf = SVC(kernel='rbf', gamma=1)
clf.fit(train_data, train_label)

predicted_label = clf.predict(train_data)
print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label.flatten()))) + '%')

# Find the accuracy on Validation Dataset
predicted_label = clf.predict(validation_data)
print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label.flatten()))) + '%')

# Find the accuracy on Testing Dataset
predicted_label = clf.predict(test_data)
print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label == test_label.flatten()))) + '%')

Radial; Gamma: 1


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print("Radial; Gamma: Default")
clf = SVC(kernel='rbf')
clf.fit(train_data, train_label)

predicted_label = clf.predict(train_data)
print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label.flatten()))) + '%')

# Find the accuracy on Validation Dataset
predicted_label = clf.predict(validation_data)
print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label.flatten()))) + '%')

# Find the accuracy on Testing Dataset
predicted_label = clf.predict(test_data)
print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label == test_label.flatten()))) + '%')

In [ ]:
preds = {}

for C in [1] + list(np.arange(10, 110, 10)):
    print("Radial; C:", C)
    clf = SVC(C=C)
    clf.fit(train_data, train_label)
    
    preds[C] = {}

    predicted_label = clf.predict(train_data)
    print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label.flatten()))) + '%')
    preds[C]['train'] = 100 * np.mean((predicted_label == train_label.flatten()))

    # Find the accuracy on Validation Dataset
    predicted_label = clf.predict(validation_data)
    print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label.flatten()))) + '%')
    preds[C]['validation'] = 100 * np.mean((predicted_label == validation_label.flatten()))

    # Find the accuracy on Testing Dataset
    predicted_label = clf.predict(test_data)
    print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label == test_label.flatten()))) + '%')
    preds[C]['testing'] = 100 * np.mean((predicted_label == test_label.flatten()))